In [4]:
import pyspark

In [339]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *
from pyspark.ml.evaluation import *

In [7]:
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [8]:
spark

In [282]:
df_new = spark.read.csv('/content/train.csv')

In [283]:
df_new = spark.read.option('header' , 'true').csv('/content/train.csv')

In [284]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [285]:
type(df_new)

pyspark.sql.dataframe.DataFrame

In [286]:
df = df_new.toPandas()

In [343]:
type(df)

pandas.core.frame.DataFrame

In [287]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S


In [288]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [289]:
df_new.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [290]:
df_new.head()

Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S')

In [291]:
df_new.select( ['PassengerId' , 'Survived' , 'Embarked'] ).show(5)

+-----------+--------+--------+
|PassengerId|Survived|Embarked|
+-----------+--------+--------+
|          1|       0|       S|
|          2|       1|       C|
|          3|       1|       S|
|          4|       1|       S|
|          5|       0|       S|
+-----------+--------+--------+
only showing top 5 rows



In [292]:
df_new.select(['Survived']).show(5)

+--------+
|Survived|
+--------+
|       0|
|       1|
|       1|
|       1|
|       0|
+--------+
only showing top 5 rows



In [293]:
df_new.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [294]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [295]:
df_new.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [296]:
df_new = df_new.withColumn('Age_plus_Fare' , df_new['Age'] + df_new['Fare'])

In [297]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-------------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_plus_Fare|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|        29.25|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|     109.2833|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|       33.925|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|         88.1|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|

In [298]:
#Drop column
df_new = df_new.drop('Age_plus_Fare')

In [299]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [300]:
df_new.withColumnRenamed('PassengerId' , 'ID').show(5)

+---+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
| ID|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+---+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|  1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|  2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|  3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|  4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|  5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+---+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
only showing top 5 rows



In [301]:
df_new.na.drop().show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male| 54|    0|    0|   17463|51.8625|  E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|  4|    1|    1| PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female| 58|    0|    0|  113783|  26.55| C103|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
only showing top 5 rows



In [344]:
df_new.na.drop(how = 'any').show(5)

+--------+---+----+----------+------------+------------+---------------+-------------+-------------+---------------+-------------+-------------+
|Survived|Age|Fare|Sex_encode|SibSp_encode|Parch_encode|Embarked_encode|   Sex_onehot| Parch_onehot|Embarked_onehot| SibSp_onehot|Pclass_onehot|
+--------+---+----+----------+------------+------------+---------------+-------------+-------------+---------------+-------------+-------------+
|       0| 22|   7|       0.0|         1.0|         0.0|            0.0|(1,[0],[1.0])|(6,[0],[1.0])|  (2,[0],[1.0])|(6,[1],[1.0])|    (3,[],[])|
|       1| 38|  71|       1.0|         1.0|         0.0|            1.0|    (1,[],[])|(6,[0],[1.0])|  (2,[1],[1.0])|(6,[1],[1.0])|(3,[1],[1.0])|
|       1| 26|   7|       1.0|         0.0|         0.0|            0.0|    (1,[],[])|(6,[0],[1.0])|  (2,[0],[1.0])|(6,[0],[1.0])|    (3,[],[])|
|       1| 35|  53|       1.0|         1.0|         0.0|            0.0|    (1,[],[])|(6,[0],[1.0])|  (2,[0],[1.0])|(6,[1],[1.0])|

In [303]:
df_new.select( mean('Age').alias('avg_age') ).show()

+-----------------+
|          avg_age|
+-----------------+
|29.69911764705882|
+-----------------+



In [304]:
df_new = df_new.na.fill(  'Unknown' ,  ['Cabin']  )

In [305]:
df_new = df_new.withColumn("Age" , col("Age").cast("int") )
df_new = df_new.withColumn("Fare" , col("Fare").cast("int"))
df_new = df_new.withColumn("Pclass" , col("Pclass").cast("int"))
df_new = df_new.withColumn("Survived" , col("Survived").cast("int"))

In [306]:
df_new.dtypes

[('PassengerId', 'string'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'int'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'int'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [309]:
df_new.select([ count(when(col(c).isNull(), c)).alias(c) for c in df_new.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|    0|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [310]:
from pyspark.ml.feature import Imputer
imputer_numeric = Imputer(
    inputCols = ['Age'] , 
    outputCols = ['{}'.format(c) for c in ['Age']]
).setStrategy("mean")

In [313]:
df_new = imputer_numeric.fit(df_new).transform(df_new)

In [314]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|Fare|  Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7|Unknown|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|  71|    C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|   7|Unknown|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|  53|   C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8|Unknown|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-

In [317]:
df_new = df_new.na.drop()

In [318]:
df_new.select([ count(when(col(c).isNull(), c)).alias(c) for c in df_new.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [319]:
df_encode = StringIndexer(inputCols=['Sex' , 'SibSp' , 'Parch' , 'Embarked'] , outputCols= ['Sex_encode' , 'SibSp_encode' , 
                                                                                               'Parch_encode' , 'Embarked_encode'] )
df_new = df_encode.fit(df_new).transform(df_new)

In [320]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-------+--------+----------+------------+------------+---------------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|Fare|  Cabin|Embarked|Sex_encode|SibSp_encode|Parch_encode|Embarked_encode|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-------+--------+----------+------------+------------+---------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7|Unknown|       S|       0.0|         1.0|         0.0|            0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|  71|    C85|       C|       1.0|         1.0|         0.0|            1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|   7|Unknown|       S|       1.0|         0.0|         0.0|            0.0|
|   

In [321]:
df_onehot = OneHotEncoder(inputCols = ['Sex_encode' , 'SibSp_encode' , 'Parch_encode' , 'Embarked_encode' , 'Pclass'] , 
                          outputCols = ['Sex_onehot' , 'SibSp_onehot' , 'Parch_onehot' , 'Embarked_onehot' , 'Pclass_onehot'])
df_new = df_onehot.fit(df_new).transform(df_new)

In [322]:
df_new.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-------+--------+----------+------------+------------+---------------+-------------+-------------+---------------+-------------+-------------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|Fare|  Cabin|Embarked|Sex_encode|SibSp_encode|Parch_encode|Embarked_encode|   Sex_onehot| Parch_onehot|Embarked_onehot| SibSp_onehot|Pclass_onehot|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-------+--------+----------+------------+------------+---------------+-------------+-------------+---------------+-------------+-------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7|Unknown|       S|       0.0|         1.0|         0.0|            0.0|(1,[0],[1.0])|(6,[0],[1.0])|  (2,[0],[1.0])|(6,[1],[1.0])|    (3,[],[])|
|          2|       1|     1|Cumings, Mrs. Joh..

In [323]:
drop_col = ["PassengerId" , "Pclass" , "Sex" , "SibSp" , "Parch" , "Ticket" , "Cabin" , "Embarked" , "Name"]
df_new = df_new.drop(*drop_col)

In [324]:
df_new.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Sex_encode: double (nullable = false)
 |-- SibSp_encode: double (nullable = false)
 |-- Parch_encode: double (nullable = false)
 |-- Embarked_encode: double (nullable = false)
 |-- Sex_onehot: vector (nullable = true)
 |-- Parch_onehot: vector (nullable = true)
 |-- Embarked_onehot: vector (nullable = true)
 |-- SibSp_onehot: vector (nullable = true)
 |-- Pclass_onehot: vector (nullable = true)



In [325]:
df_new.columns

['Survived',
 'Age',
 'Fare',
 'Sex_encode',
 'SibSp_encode',
 'Parch_encode',
 'Embarked_encode',
 'Sex_onehot',
 'Parch_onehot',
 'Embarked_onehot',
 'SibSp_onehot',
 'Pclass_onehot']

In [326]:
data_transformer = VectorAssembler(inputCols= ['Age','Fare','Sex_encode','SibSp_encode','Parch_encode','Embarked_encode','Sex_onehot',
 'Parch_onehot','Embarked_onehot','SibSp_onehot','Pclass_onehot'] , outputCol = "Independent_features")

In [327]:
data = data_transformer.setHandleInvalid("skip").transform(df_new)

In [328]:
final_data = data.select("Independent_features" , "Survived")

In [329]:
train , test = final_data.randomSplit([0.8 , 0.2])

In [331]:
train.show(5)

+--------------------+--------+
|Independent_features|Survived|
+--------------------+--------+
|(24,[0,1,2,3,4,5,...|       0|
|(24,[0,1,2,3,4,5,...|       1|
|(24,[0,1,2,3,4,5,...|       1|
|(24,[0,1,2,3,4,5,...|       1|
|(24,[0,1,2,3,4,5,...|       0|
+--------------------+--------+
only showing top 5 rows



In [332]:
lr = LogisticRegression(featuresCol= "Independent_features" , labelCol= "Survived")
lr = lr.fit(train)

In [334]:
pred_results = lr.evaluate(test)

In [335]:
pred_results.predictions.show()

+--------------------+--------+--------------------+--------------------+----------+
|Independent_features|Survived|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|       1|[-1.4054563712778...|[0.19695169514246...|       1.0|
|(24,[0,1,2,3,4,5,...|       1|[-3.2650006981545...|[0.03679158366459...|       1.0|
|(24,[0,1,2,3,4,5,...|       1|[-3.7231958017385...|[0.02358686499711...|       1.0|
|(24,[0,1,2,3,4,8,...|       1|[-2.1432004133222...|[0.10496833039343...|       1.0|
|(24,[0,1,2,3,4,8,...|       1|[-2.0296695248175...|[0.11612283718495...|       1.0|
|(24,[0,1,2,3,4,8,...|       1|[-3.1033497882097...|[0.04296929074423...|       1.0|
|(24,[0,1,2,3,4,8,...|       1|[-1.9680929169022...|[0.12259387388357...|       1.0|
|(24,[0,1,2,3,4,8,...|       0|[1.94598816571698...|[0.87500853282272...|       0.0|
|(24,[0,1,2,3,4,8,...|       0|[1.94598816571698...|[0.8750085328

In [341]:
pred_results.accuracy

0.7741935483870968

In [342]:
pred_results.areaUnderROC

0.8242408192090395

In [345]:
pred_results.weightedPrecision

0.7734997649163446